### TPC-DS Query Stream Generation  
Use dsqgen to compile SQL for a target system.  

In [1]:
import config, ds_setup

In [2]:
from importlib import reload

#### 1. Single Query Generation  

In [3]:
p = 3
scale = 100
tpl_dir = config.fp_query_templates + config.sep + "bq_ds"

query_text = ds_setup.qgen_template(n=p, 
                                    templates_dir=tpl_dir,
                                    scale=scale, 
                                    qual=None,
                                    verbose=True)
print("QUERY TEXT")
print("==========")
print(query_text)

TPC-DS dsqgen parameters
command & kwargs: ['./dsqgen', '-VERBOSE', '-SCALE', '100', '-RNGSEED', '13', '-TEMPLATE', 'query3.tpl', '-FILTER', '-DIALECT', 'sqlserver_bq', '-DIRECTORY', '/home/colin/code/bq_snowflake_benchmark/tpl/bq_ds']
cwd: /home/colin/code/bq_snowflake_benchmark/ds/v2.11.0rc2/tools

QUERY: 3

Standard Out:

select  dt.d_year 
       ,item.i_brand_id brand_id 
       ,item.i_brand brand
       ,sum(ss_ext_discount_amt) sum_agg
 from  date_dim dt 
      ,store_sales
      ,item
 where dt.d_date_sk = store_sales.ss_sold_date_sk
   and store_sales.ss_item_sk = item.i_item_sk
   and item.i_manufact_id = 492
   and dt.d_moy=11
 group by dt.d_year
      ,item.i_brand
      ,item.i_brand_id
 order by dt.d_year
         ,sum_agg desc
         ,brand_id
 limit 100;




Error Out
qgen2 Query Generator (Version 2.11.0)
Copyright Transaction Processing Performance Council (TPC) 2001 - 2019
Parsed 1 templates


QUERY TEXT

select  dt.d_year 
       ,item.i_brand_id brand_id 
      

#### 2. Query Stream Generation  

In [4]:
reload(ds_setup)
p = 3
scale = 100
tpl_dir = config.fp_query_templates + config.sep + "bq_ds"

query_text = ds_setup.qgen_stream(p=p,
                                  templates_dir=tpl_dir,
                                  scale=scale,
                                  dialect="sqlserver_bq",
                                  qual=None,
                                  verbose=True)
print("QUERY STREAM TEXT")
print("=================")
print(query_text)

TPC-DS dsqgen parameters
command & kwargs: ['./dsqgen', '-VERBOSE', '-OUTPUT_DIR', '/home/colin/code/bq_snowflake_benchmark/ds/temp_queries', '-STREAM', '4', '-INPUT', '/home/colin/code/bq_snowflake_benchmark/tpl/bq_ds/templates.lst', '-SCALE', '100', '-RNGSEED', '13', '-DIALECT', 'sqlserver_bq', '-DIRECTORY', '/home/colin/code/bq_snowflake_benchmark/tpl/bq_ds']
cwd: /home/colin/code/bq_snowflake_benchmark/ds/v2.11.0rc2/tools

QUERY STREAM: 3

Standard Out:
No Standard Output.

Error Out
qgen2 Query Generator (Version 2.11.0)
Copyright Transaction Processing Performance Council (TPC) 2001 - 2019
Parsed 99 templates


QUERY STREAM TEXT

select  *
from(
select i_category, i_class, i_brand,
       s_store_name, s_company_name,
       d_moy,
       sum(ss_sales_price) sum_sales,
       avg(sum(ss_sales_price)) over
         (partition by i_category, i_brand, s_store_name, s_company_name)
         avg_monthly_sales
from item, store_sales, date_dim, store
where ss_item_sk = i_item_sk and
   